In [8]:
import os
import pickle
import sys
import time
import eval_script as ev
import networkx as nx
import numpy as np
import ast

In [3]:
with open('datasets/pareto_front_osdpm.pickle', 'rb') as f:
        pareto_front = pickle.load(f)
with open('datasets/graph_dict_osdpm.pickle', 'rb') as f:
        graph = pickle.load(f)

In [ ]:
file_path = 'solutions/solutions_instance_0.txt'  # Replace with your file path
solutions = []


# Open the file and process line by line
with open(file_path, 'r') as file:
    lines = file.readlines()  # Read all lines into a list
for line in lines:
    line = line.strip()  # Clean whitespace

    # Safely parse the tuple/list structure
    try:
        parsed_data = ast.literal_eval(line)
    except (ValueError, SyntaxError) as e:
        print(f"Skipping line due to parsing error: {e}")
        continue

    if not (isinstance(parsed_data, tuple) and len(parsed_data) == 2):
        print(f"Skipping malformed line: {parsed_data}")
        continue

    sol_cost, sol_path = parsed_data

    solutions.append((sol_cost, sol_path))


In [10]:
pareto_found = 0
total_time = 0
exact_path_matches = 0

for instance in pareto_front:

    undominated_not_in_pareto = []
    valid_solutions= []
    invalid_paths = []

    node_a = instance['source']
    node_b = instance['target']
    pareto_set = instance['pareto_set']

    # extract pareto costs and paths
    pareto_costs = [cost for cost, _ in pareto_set]
    pareto_paths = [path for _, path in pareto_set]

    # run algorithm and time it
    #start_time = time.time()
    #solutions = algorithm(graph, node_a, node_b)
    #elapsed = time.time() - start_time
    #total_time += elapsed

    for sol_cost, sol_path in solutions:

        # check if path is valid and adheres to graph structure
        if not ev.is_valid_path(graph, sol_path):
            invalid_paths.append(sol_path)
            continue
        valid_solutions.append((sol_cost, sol_path))

        # recalculate costs to validate cost matches graph data
        calc_costs = [0, 0, 0, 0]
        valid = True

        for i in range(len(sol_path) - 1):
            u, v = sol_path[i], sol_path[i + 1]
            if v not in graph.get(u, {}):
                valid = False
                break
            edge_costs = graph[u][v]  
            for j in range(4):
                calc_costs[j] += edge_costs[j]

        if not valid or not all(abs(calc_costs[i] - sol_cost[i]) < 1e-5 for i in range(4)):
            continue 

        # path match check
        if sol_path in pareto_paths:
            exact_path_matches += 1

        # check for exact cost match 
        if sol_cost in pareto_costs:
            pareto_found += 1
        else:
            # dominance check
            undominated = True
            for pareto_cost in pareto_costs:
                if all(p <= s for p, s in zip(pareto_cost, sol_cost)) and any(p < s for p, s in zip(pareto_cost, sol_cost)):
                    undominated = False
                    break

            if undominated:
                pareto_found += 1
                # dump solutions not found in pareto front but undominated
                undominated_not_in_pareto.append((sol_cost, sol_path))


In [12]:
len(undominated_not_in_pareto)

1000

In [13]:
exact_path_matches

1